<div style="text-align: center;">
<a target="_blank" href="https://colab.research.google.com/github/bmalcover/aa_2425/blob/main/10_VGG_Custom_Dataset/VGG_Custom.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
</div>

In [1]:
import os

import torch
from torch import nn
import torch.optim as optim
from tqdm.auto import tqdm

from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET

import cv2

# Introducció

En aquesta pràctica treballarem amb l'arquitectura ResNet (Residual Network), centrant-nos en el concepte clau dels blocs residuals, que ja hem explorat a classe. Recordem breument la principal innovació d'aquestes: el bloc residual. Aquest introdueix una connexió directa (*shortcut connection*) entre l'entrada i la sortida d'un grup de capes, tal com es representa a continuació:

$$
\mathbf{y} = \mathcal{F}(\mathbf{x}, \{W_i\}) + \mathbf{x}
$$

On:
- $\mathbf{x}$ és l'entrada del bloc.
- $\mathcal{F}(\mathbf{x}, \{W_i\})$ és la transformació no lineal aplicada per les capes internes (normalment convolucions, normalització i ReLU).
- $\mathbf{y}$ és la sortida, que combina l'entrada original xx amb la transformació.

Aquest simple però efectiu mecanisme permet que el model "aprengui" només la diferència (o residual) entre l'entrada i la sortida esperada, en lloc de l'objectiu complet. Això facilita l'entrenament i redueix el risc de degradació del rendiment en xarxes profundes.

A classe hem vist que aquests blocs residuals són la base de diferents variants de ResNet, com ara ResNet-18, ResNet-50, etc., amb diferències en la profunditat i el nombre de blocs. Ara posarem en pràctica aquest coneixement treballant amb ResNets per a tasques de classificació d’imatges

# Cream mòduls amb Pytorch

En aquesta pràctica aprendrem a definir mòduls personalitzats en PyTorch utilitzant la classe base ``torch.nn.Module``. Aquesta classe permet encapsular les capes i operacions d’una xarxa neuronal, facilitant-ne la reutilització i el manteniment.

**Pasos per crear un mòdul en Pytorch:**
1. **Heretar de nn.Module**. Tots els mòduls personalitzats en PyTorch han de derivar de la classe base torch.nn.Module.
2. **Definir les capes en el constructor __init__.** Al constructor del mòdul (__init__), s’han d’inicialitzar les capes que s’utilitzaran, com ara convolucions, capes lineals o funcions d’activació.
3. **Implementar la funció forward.** Aquesta funció defineix com flueixen les dades a través del mòdul. Aquí s’apliquen les capes definides al constructor de manera seqüencial o segons sigui necessari.


## Cream un bloc residual

**Revisar sessió teòrica**

El nostre bloc residual tendrà dues capes convolucionals, batch norm i emprarà ReLU.

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            padding='same',
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=(3,3),
            padding='same',
            bias=False
        )

        self.bn2 = nn.BatchNorm2d(out_channels)
        self.adjust_input = nn.Conv2d(in_channels,out_channels,kernel_size=1,bias=False) if in_channels != out_channels else None

    def forward(self, x):
        identitat = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)

        if self.adjust_input is not None:
          identitat = self.adjust_input(identitat)

        x = x + identitat
        x = self.relu(x)

        return x

## Una VGG16 residual

Una vegada el tenim implementat farem dos models:
1. Model VGG16 **normal**. Un model com el que vàrem veure la setmana passada sense preentranement.
2. Model VGG16 **ampliat**. Heu d'afegir a una VGG16 dos blocs residuals al final de l'extractor de característiques. Per fer-ho s'ha d'emprar la mateixa estratègia que heu vist a les sessions anteriors per fer fine-tunning.

Entrena ambdós models amb el conjunt de dades [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), i contesta a la següent pregunta:

- És el mateix resultat una xarxa que l'altra? Si és així o no diguès per què?

In [10]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split

# Define the transform to resize and convert to 3 channels
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to 32x32
    transforms.ToTensor(),          # Convert to tensor
    transforms.Lambda(lambda x: x.repeat(3, 1, 1))  # Repeat to 3 channels
])

# Download Fashion MNIST dataset
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# Split the train dataset into train and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


Train dataset size: 48000
Validation dataset size: 12000
Test dataset size: 10000


In [7]:
batch_size = 4
num_epochs = 5

# Model definition (VGG16 from scratch)
vgg16 = models.vgg16(pretrained=False)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modify the classifier for 10 classes (Fashion MNIST)
num_ftrs = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(num_ftrs, 10)

vgg16 = vgg16.to(device)



# Define the loss function and optimizer
loss_fn = nn.BCEWithLogitsLoss()
learning_rate = 1e-3  # Hiperparàmetre
optimizer = optim.Adam(vgg16.parameters(), lr=learning_rate)


from sklearn.metrics import accuracy_score

running_loss = []
running_acc = []

running_test_loss = []
running_test_acc_cnn = []

for t in tqdm(range(num_epochs), desc="Èpoques"):
    batch_loss = 0
    batch_acc = 0

    i_batch = 1
    # Iteram els batches.
    for i_batch, (x, y) in tqdm(enumerate(train_loader), desc=f"Batches (Època {t + 1})"):
        vgg16.train()  # Posam el model a mode entranament.

        optimizer.zero_grad()

        # 1. PREDICCIÓ
        y_pred = vgg16(x.to(device))

        # 2. CALCUL DE LA PÈRDUA
        # Computa la pèrdua: l'error de predicció vs el valor correcte
        # Es guarda la pèrdua en un array per futures visualitzacions

        y = y.to(device)

        loss = loss_fn(y_pred, y)

        #3. GRADIENT
        vgg16.zero_grad()
        loss.backward()

        # Actualitza els pesos utilitzant l'algorisme d'actualització
        #4. OPTIMITZACIO
        with torch.no_grad():
            optimizer.step()

        # 5. EVALUAM EL MODEL
        vgg16.eval()  # Mode avaluació de la xarxa

        y_pred = vgg16(x.to(device))
        batch_loss += (loss_fn(y_pred, y).detach())
        y_pred_class = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
        batch_acc += accuracy_score(y.detach().cpu().numpy(), y_pred_class)


    running_loss.append(batch_loss / (i_batch + 1))
    running_acc.append(batch_acc / (i_batch + 1))

    batch_test_loss = 0
    batch_test_acc = 0

    vgg16.eval()
    for i_batch, (x, y) in enumerate(test_loader):
        y_pred = vgg16(x.to(device))
        batch_test_loss += (loss_fn(y_pred, y.to(device)).detach())

        y_pred_class = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
        batch_test_acc += accuracy_score(y.detach().cpu().numpy(), y_pred_class)


    running_test_loss.append(batch_test_loss / (i_batch + 1))
    running_test_acc_cnn.append(batch_test_acc / (i_batch + 1))




Èpoques:   0%|          | 0/5 [00:00<?, ?it/s]

Batches (Època 1): 0it [00:00, ?it/s]

ValueError: Target size (torch.Size([4, 1])) must be the same as input size (torch.Size([4, 10]))